In [ ]:
# default_exp slideshow

# slideshow

> Command line script to show images.

In [ ]:
#hide
from nbdev.showdoc import *

In [ ]:
#export
import logging
from glob import glob
import os.path
import time
import itertools

import cv2
import wx
import numpy as np
from tqdm import tqdm
from fastcore.script import *

In [ ]:
#export
def array_to_wx(image, size):
    height, width = image.shape[:-1]
    pad_h = max(0, size[1]-height)//2
    pad_w = max(0, size[0]-width)//2
    image = image[:,:,::-1]
    buffer = image.tobytes()
    bitmap = wx.Bitmap.FromBuffer(width, height, buffer)
    return bitmap, pad_h, pad_w

In [ ]:
#export
class Panel(wx.Panel):
    def __init__(self, parent, imgs, size, sleep):
        super(Panel, self).__init__(parent, -1)
        self.imgs = imgs
        self.size = size
        self.sleep = sleep
        self.black = np.zeros([size[0], size[0], 3], dtype=np.uint8)
        self.white = 255 * np.ones([640, 640, 3]).astype(np.uint8)
        self.parent = parent
        self.SetSize(size)
        self.SetBackgroundStyle(wx.BG_STYLE_CUSTOM)
        self.SetBackgroundColour((0,0,0))
        self.Bind(wx.EVT_PAINT, self.on_paint)
        self.frame_n = None
        self.play = False
        self.exit = False
        self.Bind(wx.EVT_KEY_DOWN, self.onKeyPress)
        self.update()
        self.parent.SetPosition((0,0))

    def update(self):
        if self.play:
            self.Refresh()
            self.Update()
        if not self.exit:
            wx.CallLater(1, self.update)

    def create_bitmap(self):
        try:
            self.frame_n, frame_name = self.imgs.__next__()
            if frame_name!='black':
                logging.debug('iter: %s, image: %s', self.frame_n//2, frame_name)
            if frame_name == 'black':
                frame = self.black
            elif frame_name == 'white':
                frame = self.white
            else:
                frame = cv2.imread(frame_name)
            bitmap, pad_h, pad_w = array_to_wx(frame, self.size)
        except StopIteration:
            frame = self.black
            frame_name = 'black'
            bitmap, pad_h, pad_w = array_to_wx(frame, self.size)
        return bitmap, pad_h, pad_w, frame_name

    def on_paint(self, event):
        bitmap, pad_h, pad_w, frame_name = self.create_bitmap()
        dc = wx.AutoBufferedPaintDC(self)
        dc.DrawBitmap(bitmap, pad_w, pad_h,  useMask=False)
        if frame_name=='black':
            time.sleep(self.sleep)
        else:
            time.sleep(0.1)

    def onKeyPress(self, event):
        keycode = event.GetKeyCode()
        if keycode == wx.WXK_SPACE:
            self.play = not(self.play)
        if keycode == wx.WXK_ESCAPE:
            self.exit = not(self.exit)
            logging.debug('Exit app')
            self.Close()
            self.parent.Close()

In [ ]:
#export
class Frame(wx.Frame):
    def __init__(self, imgs, size, sleep):
        d0w, d0h = wx.DisplaySize()
        style = wx.STAY_ON_TOP
        super(Frame, self).__init__(None, -1, 'coco imgs', style=style,
                                    size=(10,10))
        panel = Panel(self, imgs, size, sleep)
        self.Fit()
        self.SetPosition((d0w,0))

In [ ]:
#export
@call_parse
def slideshow(folder:Param("Path to the video", str),
              size:Param("Size of the window", int, nargs=2),
              sleep:Param("Presentation time for each frame. 0.1 yields 3 frames per image, 0.3 yoelds 9. (if you are filming at 30fps)", float)=0.1):
    """Slide Show.

    Example:
        slideshow path_to_folder_with_imgages 1000 1000

    Author: Andres Babino <ababino@gmail.com>
    """
    logging.basicConfig(format='%(levelname)s: %(asctime)s %(message)s', filename="stw.log", level=logging.DEBUG)
    path = os.path.join(folder, '*.jpg')
    path = ['white']+glob(path)+['white']
    black_imgs = itertools.repeat('black')
    imgs = zip(black_imgs, path)
    imgs = itertools.chain.from_iterable(imgs)
    imgs = enumerate(tqdm(imgs, total=len(path)*2, unit='images'))
    
    logging.debug('Start app')
    app = wx.App()
    frame = Frame(imgs, size, sleep)
    frame.Show()
    app.MainLoop()

In [ ]:
from nbdev.export import notebook2script; notebook2script()

Converted 00_core.ipynb.
Converted 01_dataset.ipynb.
Converted 02_slideshow.ipynb.
Converted index.ipynb.
